In [1]:
import numpy as np
import specpy
from itertools import pairwise, product

def divide_into_tiles(img, num_divisions=2, threshold=0, min_size_fraction=0.05):

    # num_divisions can be sequence (per dimension)
    # if only a scalar is given, we re-use it for all dimensions 
    if np.isscalar(num_divisions):
        num_divisions = [num_divisions] * img.ndim

    # get all candidate bboxes
    bboxes = []
    start_end_per_dimension = [map(list, pairwise(np.linspace(0, s, n+1))) for s,n in zip(img.shape, num_divisions)]
    for bbox in map(list, product(*start_end_per_dimension)):
        bboxes.append(np.array(bbox).T)

    # select only the ones with enough intensity in img
    bboxes_above_thresh = []
    for bbox in bboxes:
        # select tile defined by bbox rounded to nearest pixel
        start, end = bbox
        start = np.round(start).astype(int)
        end = np.round(end).astype(int)
        tile = img[tuple(slice(s,e) for s,e in zip(start, end))]
        # if fraction of pixels above threshold is above minimal size fraction,
        # accept into final list of bboxes
        if (tile >= threshold).sum() / tile.size >= min_size_fraction:
            bboxes_above_thresh.append(bbox.ravel())

    return bboxes_above_thresh

In [ ]:
im = specpy.get_application()
img = im.active_measurement().stack(0).data().squeeze()

divide_into_tiles(img, threshold=4, num_divisions=2)

In [ ]:
from autosted import AcquisitionPipeline
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader, FOVSettingsGenerator
from autosted.detection.roi_detection import ROIDetectorWrapper
from autosted.utils.dict_utils import get_parameter_value_array_from_dict
from autosted.utils.parameter_constants import PIXEL_SIZE_PARAMETERS

In [ ]:
# get current measurement from Imspector
im = specpy.get_application()
params = im.value_at("", specpy.ValueTree.Measurement).get()

# get pixel size of active measurement
pixel_size = get_parameter_value_array_from_dict(params, PIXEL_SIZE_PARAMETERS)

pipeline = AcquisitionPipeline("acquisition_data/pyramid", ["level0", "level1"])

# initial image at double pixel size (i.e. half resolution)
start_callback = AcquisitionTaskGenerator(
    "level0",
    JSONSettingsLoader(params),
    FOVSettingsGenerator(pixel_sizes=pixel_size*2)
)

# get ROIs to image at level1 with original pixel size using divide_into_tiles
tile_callback = AcquisitionTaskGenerator(
    "level1",
    JSONSettingsLoader(params),
    ROIDetectorWrapper(divide_into_tiles, detection_kwargs={"threshold": 4})
)

pipeline.add_callback(tile_callback, "level0")
pipeline.run(start_callback)

In [ ]:
im = specpy.get_application()
params = im.value_at("", specpy.ValueTree.Measurement).get()

pixel_size = get_parameter_value_array_from_dict(params, PIXEL_SIZE_PARAMETERS)

pipeline = AcquisitionPipeline("acquisition_data/pyramid", ["level0", "level1", "level2"])

# by default, increasing levels have a lower priority number, i.e. they will be imaged first
# by giving them increasing priorities, we can instead do a breadth-first traversal
pipeline.level_priorities = {"level0": 0, "level1": 1, "level2": 2}

# first overview at 9-fold subsampling
start_callback = AcquisitionTaskGenerator(
    "level0",
    JSONSettingsLoader(params),
    FOVSettingsGenerator(pixel_sizes=pixel_size*9)
)

# second level at 3-fold subsampling
tile_callback_l1 = AcquisitionTaskGenerator(
    "level1",
    JSONSettingsLoader(params),
    FOVSettingsGenerator(pixel_sizes=pixel_size*3),
    ROIDetectorWrapper(divide_into_tiles, detection_kwargs={"threshold": 5, "num_divisions": 3})
)

# 3rd level at original pixel size
tile_callback_l2 = AcquisitionTaskGenerator(
    "level2",
    JSONSettingsLoader(params),
    ROIDetectorWrapper(divide_into_tiles, detection_kwargs={"threshold": 5, "num_divisions": 3})
)

pipeline.add_callback(tile_callback_l1, "level0")
pipeline.add_callback(tile_callback_l2, "level1")
pipeline.run(start_callback)

In [ ]:
from autosted.callback_buildingblocks.parameter_filtering import LocationRemover

pipeline = AcquisitionPipeline("acquisition_data/pyramid", ["level0", "level1"])

start_callback = AcquisitionTaskGenerator(
    "level0",
    LocationRemover(JSONSettingsLoader("config_json/test_overview.json")),
)

tile_callback = AcquisitionTaskGenerator(
    "level1",
    LocationRemover(JSONSettingsLoader("config_json/test_detail.json")),
    ROIDetectorWrapper(divide_into_tiles, detection_kwargs={"threshold": 4, "num_divisions": 12})
)

pipeline.add_callback(tile_callback, "level0")
pipeline.run(start_callback)

In [12]:
from autosted.utils.coordinate_utils import approximate_pixel_shift_from_settings
from calmutils.stitching.transform_helpers import translation_matrix
from calmutils.stitching.fusion import fuse_image

level = "level1"
channel = 1
configuration = 0

# get all measurement settings and images at selected level, config and channel
idx_len = pipeline.hierarchy_levels.index(level) + 1
settings = [measurement.measurement_settings[configuration] for idx, measurement in pipeline.data.items() if len(idx) == idx_len]
images = [measurement.data[configuration][channel].squeeze() for idx, measurement in pipeline.data.items() if len(idx) == idx_len]

# get pixel shifts of all images relative to first
pixel_shifts = [approximate_pixel_shift_from_settings(settings[0], setting_i) for setting_i in settings]

# to transformation matrix
is2d = images[0].ndim == 2
transforms = [translation_matrix(shift[(1 if is2d else 0):]) for shift in pixel_shifts]

# fuse (low out-of-bounds value to better visualize non-imaged areas)
fused = fuse_image(images, transforms, oob_val=-5)

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(fused, cmap="magma", clim=(-5, 25))